In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd
from collections import Counter
import numpy as np
import missingno
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import torch
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import dgl
import torch
import torch.nn.functional as F
from torch_sparse import SparseTensor,coalesce
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

2023-07-20 19:10:24.290520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 19:10:25.006809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv('ctr_data_1M_modified.csv',index_col=0)
data

,user_id,item_id,click,follow,like,share,video_category,watching_times,gender,age,hist_1,hist_2,hist_3,hist_4,hist_5,hist_6,hist_7,hist_8,hist_9,hist_10
0,1,4,0,0,0,0,1,0,1,4,2,3,80936,781,111774,1230,26403,991,2362,1202
1,1,1201,1,0,0,0,1,1,1,4,2,3,80936,781,111774,1230,26403,991,2362,1202
2,1,250502,1,0,0,0,1,1,1,4,2,3,80936,781,111774,1230,26403,991,2362,1202
3,1,50885,1,0,0,0,1,1,1,4,2,3,80936,781,111774,1230,26403,991,2362,1202
4,1,16934,1,0,0,0,0,1,1,4,2,3,80936,781,111774,1230,26403,991,2362,1202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115787993,961527,1364827,0,0,0,0,1,0,2,2,1466383,1380307,1383378,1675473,1354652,1354559,1369848,0,0,0
115787994,961527,1352546,0,0,0,0,0,1,2,2,1466383,1380307,1383378,1675473,1354652,1354559,1369848,0,0,0
115787995,961527,1377052,0,0,0,0,0,0,2,2,1466383,1380307,1383378,1675473,1354652,1354559,1369848,0,0,0
115787996,961527,1684451,0,0,0,0,0,0,2,2,1466383,1380307,1383378,1675473,1354652,1354559,1369848,0,0,0


In [3]:
sample_df = data.sample(n=10000, random_state=40)
sample_df

,user_id,item_id,click,follow,like,share,video_category,watching_times,gender,age,hist_1,hist_2,hist_3,hist_4,hist_5,hist_6,hist_7,hist_8,hist_9,hist_10
45200389,324033,15467,0,0,0,0,0,1,2,3,8168,127570,80817,8017,1466,69636,20083,16447,7012,74738
92020927,711789,44739,0,0,0,0,0,1,1,3,112546,30157,35716,80320,254641,21485,28030,56608,3098,201941
82905274,637536,15281,0,0,0,0,0,1,1,2,285883,3827,115814,4850,231462,347157,789,17254,300,6805
70828031,534240,10794,1,0,0,0,1,2,1,2,162208,72961,839,18093,91614,2596,640,2248,889,1850
40139660,283107,225161,0,0,0,0,1,0,1,2,1767,8912,390876,353531,25434,26105,36917,237561,6137,242420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21929340,155984,5271,0,0,0,0,0,1,2,2,1264,433,834,3682,11307,1190,24577,6181,23865,14896
92477065,715617,4846,0,0,0,0,1,0,1,2,67143,1141,183750,30075,52295,53651,30544,37996,2885,6299
83807677,644889,7155,0,0,0,0,0,1,2,3,482,6379,7070,1280,3118,2362,7183,55564,65524,8923
8049303,59040,12386,1,0,0,0,0,1,2,2,165609,38748,975,5233,111169,2556,4537,140,451,46744


In [40]:
# Normalize numerical features (if needed)
numerical_features = ['watching_times', 'age', 'hist_1', 'hist_2', 'hist_3', 'hist_4', 'hist_5', 'hist_6', 'hist_7', 'hist_8', 'hist_9', 'hist_10']
sample_df[numerical_features] = (sample_df[numerical_features] - sample_df[numerical_features].min()) / (sample_df[numerical_features].max() - sample_df[numerical_features].min())

# Create mappings for user and item IDs to contiguous integer values
user_id_map = {id: idx for idx, id in enumerate(sample_df["user_id"].unique())}
item_id_map = {id: idx for idx, id in enumerate(sample_df["item_id"].unique())}

# Map user and item IDs in the data to contiguous integer values
sample_df["user_id"] = sample_df["user_id"].map(user_id_map)
sample_df["item_id"] = sample_df["item_id"].map(item_id_map)

# Total number of users and items
num_users = len(user_id_map)
num_items = len(item_id_map)

# Create user-item interaction matrices (user-item adjacency matrix)
user_indices = torch.tensor(sample_df["user_id"].values, dtype=torch.long)
item_indices = torch.tensor(sample_df["item_id"].values, dtype=torch.long)
values = torch.ones_like(user_indices, dtype=torch.float32)

# Construct the adjacency matrix
adj_matrix = SparseTensor(
    row=user_indices,
    col=item_indices,
    value=values,
    sparse_sizes=(num_users, num_items)
)

In [6]:
sample_df

,user_id,item_id,click,follow,like,share,video_category,watching_times,gender,age,hist_1,hist_2,hist_3,hist_4,hist_5,hist_6,hist_7,hist_8,hist_9,hist_10
45200389,324033,15467,0,0,0,0,0,0.000948,2,0.428571,0.004574,0.033509,0.021228,0.002106,0.000385,0.018192,0.005206,0.004320,0.001842,0.019572
92020927,711789,44739,0,0,0,0,0,0.000948,1,0.428571,0.063018,0.007921,0.009382,0.021098,0.066887,0.005613,0.007267,0.014869,0.000814,0.052883
82905274,637536,15281,0,0,0,0,0,0.000948,1,0.285714,0.160076,0.001005,0.030421,0.001274,0.060799,0.090694,0.000205,0.004532,0.000079,0.001782
70828031,534240,10794,1,0,0,0,1,0.001896,1,0.285714,0.090826,0.019165,0.000220,0.004753,0.024065,0.000678,0.000166,0.000590,0.000234,0.000484
40139660,283107,225161,0,0,0,0,1,0.000000,1,0.285714,0.000989,0.002341,0.102673,0.092863,0.006681,0.006820,0.009571,0.062401,0.001612,0.063484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21929340,155984,5271,0,0,0,0,0,0.000948,2,0.285714,0.000708,0.000114,0.000219,0.000967,0.002970,0.000311,0.006371,0.001624,0.006269,0.003901
92477065,715617,4846,0,0,0,0,1,0.000000,1,0.285714,0.037596,0.000300,0.048266,0.007900,0.013737,0.014016,0.007918,0.009981,0.000758,0.001650
83807677,644889,7155,0,0,0,0,0,0.000948,2,0.428571,0.000270,0.001676,0.001857,0.000336,0.000819,0.000617,0.001862,0.014595,0.017211,0.002337
8049303,59040,12386,1,0,0,0,0,0.000948,2,0.285714,0.092730,0.010178,0.000256,0.001375,0.029201,0.000668,0.001176,0.000037,0.000118,0.012241


In [44]:
# NGCF Model
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        torch.nn.init.xavier_uniform_(self.weight)
        torch.nn.init.zeros_(self.bias)

    def forward(self, adj_matrix, node_embeddings):
        support = adj_matrix.matmul(node_embeddings)
        output = torch.matmul(support, self.weight) + self.bias
        return output

class NGCF(nn.Module):
    def __init__(self, num_users, num_items, embed_size):
        super(NGCF, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embed_size)
        self.item_embeddings = nn.Embedding(num_items, embed_size)
        self.gcn_layer = GCNLayer(embed_size, embed_size)

    def forward(self, user_indices, item_indices, adj_matrix):
        user_embeds = self.user_embeddings(user_indices)
        item_embeds = self.item_embeddings(item_indices)

        # Forward pass through GCN layer
        node_embeddings = torch.cat((user_embeds, item_embeds), dim=0)
        gcn_output = self.gcn_layer(adj_matrix, node_embeddings)

        # Separate user and item embeddings using advanced indexing
        num_users = user_indices.size(0)
        num_items = item_indices.size(0)
        user_embeds = gcn_output[:num_users]
        item_embeds = gcn_output[num_users:]

        return user_embeds, item_embeds

# Negative Sampling
def negative_sampling(user_indices, num_users, num_items):
    num_negatives = len(user_indices)
    user_negatives = torch.tensor([np.random.randint(0, num_users) for _ in range(num_negatives)], dtype=torch.long)
    item_negatives = torch.tensor([np.random.randint(0, num_items) for _ in range(num_negatives)], dtype=torch.long)
    return user_negatives, item_negatives

# Training
num_epochs = 10
batch_size = 256
learning_rate = 0.001
embed_size = 64

# Data Loader
dataset = TensorDataset(user_indices, item_indices)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create NGCF model
ngcf_model = NGCF(num_users, num_items, embed_size)
optimizer = optim.Adam(ngcf_model.parameters(), lr=learning_rate)

# ... (previous code)
# Training Loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for user_batch, item_batch in train_loader:
        optimizer.zero_grad()

        # Ensure user_batch and item_batch are 1-dimensional tensors
        user_batch = user_batch.view(-1)
        item_batch = item_batch.view(-1)

        user_embeds, item_embeds = ngcf_model(user_batch, item_batch, adj_matrix)

        # Reshape user_embeds and item_embeds for multiplication
        user_embeds = user_embeds.view(-1, 1, embed_size)
        item_embeds = item_embeds.view(1, -1, embed_size)

        # Compute element-wise product and sum across embeddings to get scores
        scores = torch.matmul(user_embeds, item_embeds.transpose(1, 2)).squeeze()

        # Negative Sampling
        neg_user_batch, neg_item_batch = negative_sampling(user_batch, num_users, num_items)
        neg_user_embeds, neg_item_embeds = ngcf_model(neg_user_batch, neg_item_batch, adj_matrix)

        # Reshape neg_user_embeds and neg_item_embeds for multiplication
        neg_user_embeds = neg_user_embeds.view(-1, 1, embed_size)
        neg_item_embeds = neg_item_embeds.view(1, -1, embed_size)

        # Compute negative scores
        neg_scores = torch.matmul(neg_user_embeds, neg_item_embeds.transpose(1, 2)).squeeze()

        # Calculate BPR loss
        mse_loss = nn.MSELoss()
        loss = mse_loss(scores, neg_scores)
        #loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

Epoch 1/10, Loss: nan
Epoch 2/10, Loss: nan
Epoch 3/10, Loss: nan
Epoch 4/10, Loss: nan
Epoch 5/10, Loss: nan
Epoch 6/10, Loss: nan
Epoch 7/10, Loss: nan
Epoch 8/10, Loss: nan
Epoch 9/10, Loss: nan
Epoch 10/10, Loss: nan
